# Common Imports/Helpers

In [60]:
from collections import defaultdict
from itertools import cycle
import re
import string

# Day 1

In [ ]:
def day1_1():
    resulting_frequency = 0
    with open ('./day1/input.txt') as changes:
        for raw_change in changes:
            resulting_frequency += int(raw_change)
    return resulting_frequency

print(day1_1())

In [ ]:
def day1_2():  
    changes = []
    with open ('./day1/input.txt') as raw_changes:
        for raw_change in raw_changes:
            changes.append(int(raw_change))

    seen_frequencies = set()
    current_frequency = 0
    for change in cycle(changes):
        current_frequency += change
        if current_frequency in seen_frequencies:
            return current_frequency
        seen_frequencies.add(current_frequency)

    return current_frequency

print(day1_2())

# Day 2

In [ ]:
def day2_1():
    double_count = 0
    triple_count = 0
    with open ('./day2/input.txt') as box_ids:
        for box_id in box_ids:
            letter_counts = [box_id.count(l) for l in string.ascii_lowercase]
            if letter_counts.count(2) > 0:
                double_count += 1
            if letter_counts.count(3) > 0:
                triple_count += 1
    return double_count * triple_count

In [ ]:
print(day2_1())

In [ ]:
def day2_2():
    box_ids_without_one_letter = set()
    with open ('./day2/input.txt') as box_ids:
        for raw_box_id in box_ids:
            box_id = raw_box_id.strip()
            for i in range(len(box_id)):
                box_id_without_one_letter = box_id[:i] + '_' + box_id[i+1:]
                if box_id_without_one_letter in box_ids_without_one_letter:
                    return box_id_without_one_letter.replace('_', '')
                box_ids_without_one_letter.add(box_id_without_one_letter)

In [ ]:
print(day2_2())

# Day 3

In [ ]:
claim_regex = re.compile(r'#(\d+) @ (\d+),(\d+): (\d+)x(\d+)')
def parse_claim(raw_claim):
    return tuple(int(n) for n in claim_regex.match(raw_claim).groups())

def get_claims():
    with open ('./day3/input.txt') as claims:
        for claim in claims:
            yield parse_claim(claim.strip())

In [ ]:
def day3_1():
    grid = [[0 for y in range(10000)] for x in range(10000)]
    for _, x, y, width, height in get_claims():
        for i in range(width):
            for j in range(height):
                grid[x+i][y+j] += 1

    count = 0
    for row in grid:
        for cell in row:
            if cell > 1:
                count += 1

    return count
print(day3_1())

In [ ]:
def day3_2():
    grid = [[0 for y in range(10000)] for x in range(10000)]
    for _, x, y, width, height in get_claims():
        for i in range(width):
            for j in range(height):
                grid[x+i][y+j] += 1

    for claim_id, x, y, width, height in get_claims():
        overlaps = False
        for i in range(width):
            for j in range(height):
                if grid[x+i][y+j] > 1:
                    overlaps = True
        if not overlaps:
            return claim_id

    return None
print(day3_2())

# Day 4

In [ ]:
timestamp_regex = re.compile(r'\[1518\-(\d\d)\-(\d\d) (\d\d):(\d\d)\] (.+)')
def parse_timestamp(raw_timestamp):
    month, day, hour, minute, update = timestamp_regex.match(raw_timestamp).groups()
    return month, day, hour, minute, update

def get_updates():
    with open ('./day4.txt') as timestamps:
        for timestamp in timestamps:
            yield parse_timestamp(timestamp.strip())

def get_sorted_updates():
    return sorted(get_updates())

shift_begin_regex = re.compile(r'Guard #(\d+) begins shift')
def day4_1():
    guards = {}
    
    current_guard_id = None
    last_sleep_start = None
    for month, day, hour, minute, update in get_sorted_updates():
        if update == 'falls asleep':
            last_sleep_start = (hour, minute)
        elif update == 'wakes up':
            start_hour, start_minute = last_sleep_start
            if start_hour > hour:
                for m in range(int(start_minute), 60):
                    guards[current_guard_id][(int(start_hour), m)] += 1
                for h in range(int(start_hour) + 1, 24):
                    for m in range(60):
                        guards[current_guard_id][(h, m)] += 1
            else:
                for m in range(int(start_minute), int(minute)):
                    guards[current_guard_id][(0, m)] += 1
        else:
            current_guard_id = shift_begin_regex.match(update).group(1)
            if current_guard_id not in guards:
                guards[current_guard_id] = defaultdict(int)
    
    guard_id, sleep_counts = max(guards.items(), key=lambda x: sum(x[1].values()))
    return int(guard_id) * max(sleep_counts.keys(), key=lambda k: sleep_counts[k])[1]
            
print(day4_1())

In [ ]:
def day4_2():
    guards = {}
    
    current_guard_id = None
    last_sleep_start = None
    for month, day, hour, minute, update in get_sorted_updates():
        if update == 'falls asleep':
            last_sleep_start = (hour, minute)
        elif update == 'wakes up':
            start_hour, start_minute = last_sleep_start
            if start_hour > hour:
                for m in range(int(start_minute), 60):
                    guards[current_guard_id][(int(start_hour), m)] += 1
                for h in range(int(start_hour) + 1, 24):
                    for m in range(60):
                        guards[current_guard_id][(h, m)] += 1
            else:
                for m in range(int(start_minute), int(minute)):
                    guards[current_guard_id][(0, m)] += 1
        else:
            current_guard_id = shift_begin_regex.match(update).group(1)
            if current_guard_id not in guards:
                guards[current_guard_id] = defaultdict(int)
    guard_id, sleep_counts = max(guards.items(), key=lambda x: max(list(x[1].values()) + [0]))
    return int(guard_id) * max(sleep_counts.keys(), key=lambda k: sleep_counts[k])[1]
print(day4_2())

# Day 5

In [ ]:
def get_polymer():
    with open ('./day5.txt') as polymer:
        return polymer.read().strip()

In [ ]:
def react_polymer(p):
    while True:
        hasReacted = False
        for i in range(len(p)-1):
            a, b = p[i], p[i+1]
            if abs(ord(a) - ord(b)) == 32:
                hasReacted = True
                p = p[:i] + p[i+2:]
                break
        if not hasReacted:
            break
    return p

In [ ]:
react_polymer(get_polymer())

In [ ]:
def strip_polymer(p, l):
    return p.replace(l.upper(), '').replace(l.lower(), '')

In [ ]:
strip_polymer('dabAcCaCBAcCcaDA', 'a')

In [ ]:
def get_shortest_polymer():
    stable = react_polymer(get_polymer())
    for l in string.ascii_lowercase:
        print(l, len(react_polymer(strip_polymer(stable, l))))


In [ ]:
get_shortest_polymer()

# Day 7

In [94]:
def get_graph():
    G = dict()
    for l in string.ascii_uppercase:
        G[l] = set()
    er = re.compile(r'Step ([A-Z]) must be finished before step ([A-Z]) can begin.')
    with open('day7.txt') as raw_edges:
        for e in raw_edges:
            u, v = er.match(e).groups()
            G[v].add(u)
    return G

In [105]:
def day7_1():
    G = get_graph()
    done_steps = []
    while len(done_steps) < 26:
        for l in string.ascii_uppercase:
            if l in done_steps:
                continue
            if len(G[l] - set(done_steps)) == 0:
                done_steps.append(l)
                break
    print(''.join(done_steps))

In [106]:
day7_1()

BGKDMJCNEQRSTUZWHYLPAFIVXO


In [ ]:
def day7_2():
    G = get_graph()
    done_steps = []
    worker_progress = [0, 0, 0, 0, 0]
    while len(done_steps) < 26:
        for l in string.ascii_uppercase:
            if l in done_steps:
                continue
            if len(G[l] - set(done_steps)) == 0:
                done_steps.append(l)
                break
    print(''.join(done_steps))